## Датасет для бенчмаркинга моделей суммаризации.

Реферирование - создание краткого содержания по тексту. 

Метрики качества моделей реферирования (ROUGE, BLEU и др.) основаны на сопоставлении реферата, полученного моделью, с эталонным рефератом (то есть таким, который написал человек после прочтения исходного текста). 

Собирать датасет пар <текст, эталонный реферат> дорого!

Решение: краткие содержания c briefly.ru.

Собираем датасет пар <текст книги, краткое содержание>.


In [1]:
import os
import nltk
import pandas as pd
import numpy as np

from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser

from rouge import Rouge 

In [2]:
rouge = Rouge()
tokenizer = Tokenizer('russian')

In [3]:
luhn = LuhnSummarizer()
text_rank = TextRankSummarizer()
lex_rank = LexRankSummarizer()
lsa = LsaSummarizer()

summarizers = [luhn, text_rank, lex_rank, lsa]

In [4]:
def summarize(text, summarizer, summary_len):
#     print(summarizer.__class__, '...')
    parser = PlaintextParser.from_string(text, tokenizer)
    predicted_summary = summarizer(parser.document, summary_len)
    predicted_summary = ' '.join([str(s) for s in predicted_summary])
    return predicted_summary

В данном ноутбуке проведем бенчмаркинг моделей реферирования без учителя с помощью нашего датасета.

Будем использовать метрики качества реферирования Rouge.

Сравним результаты с Gazeta dataset (https://arxiv.org/abs/2006.11063)


### Датасет gazeta

In [5]:
gazeta_df = pd.read_json('gazeta/test.json', lines=True)

In [6]:
len(gazeta_df)

5770

In [7]:
# сгенерим саммари для датасета gazeta и вычислим для них ROUGE-метрики 

gazeta_results = [None] * len(gazeta_df)

for i in range(len(gazeta_df)): 
    text = gazeta_df.loc[i].text
    reference = gazeta_df.loc[i].summary
    ref_doc = PlaintextParser.from_string(reference, tokenizer).document
    summary_len = len(ref_doc.sentences) 
    
    result_for_text = []
    for summarizer in summarizers:
        hypothesis = summarize(text, summarizer, summary_len)
        scores = rouge.get_scores(hypothesis, reference)
        result_for_text.append([scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']])
    
    gazeta_results[i] = result_for_text
    

In [9]:
gazeta_results = np.array(gazeta_results)

In [11]:
gazeta_results.mean(axis=0)

array([[0.17066938, 0.04617214, 0.15357187],
       [0.15079518, 0.03627782, 0.1353037 ],
       [0.17230475, 0.04771969, 0.15645291],
       [0.14398278, 0.03200836, 0.12893452]])

### Наш датасет

In [12]:
books_path = 'lib/library/books/'
summaries_path = 'lib/library/summaries/'
generated_path = 'generated/'

In [13]:
# сгенерим саммари разными методами и сохраним в файлы (поскольку тексты все-таки большие)

filenames = os.listdir(books_path)

for i in range(len(filenames)):
    with open(books_path + filenames[i], 'r', encoding='utf-8') as f:
        text = f.read()
    with open(summaries_path + filenames[i], 'r', encoding='utf-8') as f:
        reference = f.read()
    ref_doc = PlaintextParser.from_string(reference, tokenizer).document
    summary_len = len(ref_doc.sentences)
#     print(i+1)
#     print(len(PlaintextParser.from_string(text, tokenizer).document.sentences), ' -> ', summary_len) 
    
    result_for_text = []
    for n, summarizer in enumerate(summarizers):
        hypothesis = summarize(text, summarizer, summary_len)
        with open(generated_path + str(n) + '/' + filenames[i], 'w') as f:
            f.write(hypothesis)    

In [14]:
# теперь получим ROUGE-метрики для сгенерированных саммари 

our_results = []
filenames = os.listdir(generated_path+'0/')
for filename in filenames:
    result_for_text = []
    with open(summaries_path + filename, 'r', encoding='utf-8') as f:
        reference = f.read()
    for i in range(4):
        with open(generated_path+str(i)+'/'+filename, 'r') as f:
            hypothesis = f.read()
            scores = rouge.get_scores(hypothesis, reference)
            result_for_text.append([scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']])
    our_results.append(result_for_text)

In [15]:
our_results = np.array(our_results)

In [18]:
our_results.mean(axis=0)

array([[0.15132326, 0.03291042, 0.14136158],
       [0.14364756, 0.03324122, 0.13419271],
       [0.13101212, 0.01841673, 0.12589934],
       [0.15114855, 0.0274097 , 0.14282233]])